## Feature Engineering with LanceDB and Geneva

This notebook will focus on the crucial process of feature engineering. We'll start with a raw dataset of fashion products, ingest it in LanceDB, then enrich our data with meaningful features that we could use to build a search engine or train a model.

We will cover the following steps:
1. **Data Ingestion**: Downloading a fashion dataset and loading it into a LanceDB table.
2. **Declarative Feature Engineering**: Using Geneva to define and compute features on-the-fly.
3. **Embedding Generation**: Creating vector embeddings for both images and text to enable semantic search.
4. **Updating**: Adding more raw data to our table and rerunning our backfills on only the new data.

## Note about Colab

This notebook runs on Google Colab, even the free tier, but it will be slow, because it has to start a local Ray cluster and execute multiple ML models on its workers. We recommend downloading this notebook and running it locally. But if you do run on Colab, we recommend:
- using a GPU instance (Runtime -> Change runtime type)
- running on only 100 rows
- not drawing conclusions about speed from this notebook. This notebook is meant as a demo of the basic workflow of feature engineering with LanceDB, not a benchmark or speed demo.

In [ ]:
!pip install --upgrade geneva lancedb pylance
!pip install kubernetes "ray[default]" rerankers pandas torch torchvision open-clip-torch transformers

# Pin protobuf to avoid MessageFactory.GetPrototype AttributeError warnings (removed in protobuf 6.30.0+)
!pip install "protobuf<6.30.0"

## 1. Data Ingestion

First, let's download our dataset. We're using a small version of the Fashion Product Images dataset from Kaggle. This dataset contains images and metadata for a variety of fashion products.

In [ ]:
#!sudo rm -r db fashion-dataset # Uncomment and run this to delete the dataset if it already exists

# Download the dataset if it doesn't exist
!test -d fashion-dataset && test -n "$(ls -A fashion-dataset 2>/dev/null)" && \
  echo "Dataset already exists, skipping download" || \
    (curl -L -o fashion-product-images-small.zip https://www.kaggle.com/api/v1/datasets/download/paramaggarwal/fashion-product-images-small \
    && unzip -q fashion-product-images-small.zip -d fashion-dataset/)

## Set Scale based on your environment

This tutorial uses Ray locally to build features, which means the scale of concurrent jobs will be limited to the system you're working on. These parameters are good defaults, but feel free to adjust them if you'd like.

In [ ]:
# Especially if on Colab, start with just 100 rows for testing.
DATASET_SIZE = 100
# Increase this if you're running locally, you have more CPUs available and want it to run faster.
CONCURRENCY = 4
CHECKPOINT_SIZE = min(300, DATASET_SIZE / 2)

In [ ]:
import io
import geneva
from geneva import udf
import lancedb
import pandas as pd
import pyarrow as pa
import numpy as np
from pathlib import Path
from PIL import Image
import torch
import open_clip
from typing import Callable
from transformers import BlipProcessor, BlipForConditionalGeneration

IMG_DIR = Path("fashion-dataset/images")
STYLE_CSV = Path("fashion-dataset/styles.csv")
DB_PATH = "./db"
TABLE_NAME = "products"


Now, let's load the data into a LanceDB table. We'll read the CSV file with the product metadata, and for each product, we'll also load the corresponding image from the `images` directory. We'll then create a LanceDB table and add the data to it in batches. LanceDB can store objects(images in this case) along with vector embeddings and metadata.

In [ ]:
# STYLE_CSV has info about the clothes: IDs, descriptions, and image paths
# Images themselves are stored in IMG_DIR; generate_rows will combine them with the metadata
# in STYLE_CSV and load them all in to our LanceDB table.
df = pd.read_csv(STYLE_CSV, on_bad_lines='skip')
df = df.dropna(subset=["id", "productDisplayName"])
df = df.drop_duplicates(subset=["id"], keep="first")

def generate_rows(df, img_dir, start=0, end=DATASET_SIZE):
    for _, row in df.iloc[start:end].iterrows():
        img_path = img_dir / f"{row['id']}.jpg"
        if not img_path.exists():
            continue
        with open(img_path, "rb") as f:
            yield {
                "id": int(row["id"]),
                "description": row["productDisplayName"],
                "image_bytes": f.read()
            }


db = lancedb.connect(DB_PATH)
# Drop the table if it already exists so we can recreate it
try:
    table = db.drop_table(TABLE_NAME)
except ValueError as e:
    pass
    
data_stream = generate_rows(df, IMG_DIR)
table = None

# Create the table and load rows in in batches
rows = []
for row in data_stream:
    rows.append(row)
    if len(rows) == min(1000, DATASET_SIZE / 2):
        if table:
            table.add(rows)
        else:
            # You have to provide schema or some data to create the table. Here we create the 
            # table with the first batch of data for simplicity.
            table = db.create_table(TABLE_NAME, data=rows)
        rows = []
if rows:
    table.add(rows)
    
len(table)

## 2. Feature Engineering with Geneva

Now that we have our data in a LanceDB table, we can start engineering features. We'll use Geneva to create new features for our products. 

### Defining geneva UDF

Geneva uses Python User Defined Functions (UDFs) to define features as columns in a Lance dataset. Adding a feature is straightforward:

1. Prototype your Python function in your favorite environment.
2. Wrap the function with small UDF decorator.
3. Register the UDF as a virtual column using Table.add_columns().
4. Trigger a backfill operation

UDFs can work on one row or a batch at a time, and can be stateful (e.g. some work is done to set up a model the first time, and future runs use the same model) or stateless. [Read more about geneva UDFs here.](https://docs.lancedb.com/geneva/udfs)

### Simple Feature Extraction

Let's start with a simple feature: extracting color tags from the product description. We'll define a User-Defined Function (UDF) that takes the product description as input and returns a comma-separated string of colors found in the description.

In [ ]:
db = geneva.connect(DB_PATH)
table = db.open_table(TABLE_NAME)

In [ ]:
@udf
def color_tags(description: str)-> str:
    colors = ["black", "white", "red", "blue", "green", "yellow", "pink", "brown", "grey", "silver"]
    return ", ".join([c for c in colors if c in description.lower()])

### Adding a Computed Column

Now that we've defined our feature-generating UDF, we can add it to our table as a computed column. Computed columns are computed on-the-fly when you perform a backfill operation.

In [ ]:
if "color_tags" in table.schema.names:
    table.drop_columns(["color_tags"])
    
table.add_columns({
    "color_tags": color_tags,
})

Let's inspect the table schema to see our newly registered UDF.

In [ ]:
table.schema

### Backfilling Features

Triggering backfill creates a distributed job to run the UDF and populate the column values in your LanceDB table. The Geneva framework simplifies several aspects of distributed execution.

Checkpoints: Each batch of UDF execution is checkpointed so that partial results are not lost in case of job failures. Jobs can resume and avoid most of the expense of having to recalculate values.

`backfill()` accepts various params to customize scale of your workload, here we'll use:

* `checkpoint_size` - the number of rows that are processed before writing a checkpoint
* `concurrency` - how many nodes are used for parallelization

Here, we'll use `db.local_ray_context()` to run on a local Ray instance, so we don't need to set up a Ray cluster, but you can also use the same setup and run distributed jobs remotely on Ray clusters.

In [ ]:
with db.local_ray_context():
    table.backfill("color_tags", checkpoint_size=CHECKPOINT_SIZE, concurrency=CONCURRENCY)

Let's take a look at our enriched data.

In [ ]:
table.search().limit(5).to_pandas()

## 3. Embedding Generation

Now that we have our text-based features, let's create some vector embeddings. Embeddings are numerical representations of data that capture its semantic meaning. We'll create embeddings for our product images and for our new `summary` and `occasion` features.

### Image Embeddings

We'll use a pretrained CLIP model to generate embeddings for our product images. We'll define a UDF that takes a batch of image bytes as input, preprocesses them, and then uses the CLIP model to generate embeddings.

In [ ]:
@udf(version="0.1", num_gpus=1 if torch.cuda.is_available() else 0, data_type=pa.list_(pa.float32(), 512))
class GenImageEmbeddings(Callable):

    def __init__(self):
        self.is_loaded=False


    def setup(self):
        self.model, _, self.preprocess = open_clip.create_model_and_transforms("ViT-B-32", pretrained="laion2b_s34b_b79k")
        self.tokenizer = open_clip.get_tokenizer("ViT-B-32")
        self.device = "cuda" if torch.cuda.is_available() else "cpu"
        self.model = self.model.to(self.device).eval()

        self.is_loaded=True

    def __call__(self, image_bytes:pa.Array) -> pa.Array:
        if not self.is_loaded:
            self.setup()

        embeddings = []
        for b in image_bytes:
            this_image_bytes = b.as_buffer().to_pybytes()

            image_stream = io.BytesIO(this_image_bytes)
            img = Image.open(image_stream).convert("RGB")
            img_tensor = self.preprocess(img).unsqueeze(0).to(self.device)
            with torch.no_grad():
                emb_tensor = self.model.encode_image(img_tensor)
                emb_tensor /= emb_tensor.norm(dim=-1, keepdim=True)
            np_emb = emb_tensor.squeeze().cpu().numpy().astype(np.float32)

            flat = pa.array(np_emb) # 1D float32 vector of shape (512,)
            embeddings.append(flat)

        stacked = pa.FixedSizeListArray.from_arrays(pa.concat_arrays(embeddings), 512)
        return stacked

In [ ]:
@udf(version="0.1", num_gpus=1 if torch.cuda.is_available() else 0, data_type=pa.string())
class GenCaptions(Callable):

    def __init__(self):
        self.is_loaded=False

    def setup(self):
        self.device = "cuda" if torch.cuda.is_available() else "cpu"
        self.processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base", use_fast=True)
        self.model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base")
        self.model = self.model.to(self.device).eval()
        self.is_loaded=True

    def __call__(self, image_bytes:pa.Array) -> pa.Array:
        if not self.is_loaded:
            self.setup()

        captions = []
        for b in image_bytes:
            this_image_bytes = b.as_buffer().to_pybytes()
            
            image_stream = io.BytesIO(this_image_bytes)
            img = Image.open(image_stream).convert("RGB")
            
            inputs = self.processor(img, return_tensors="pt").to(self.device)
            # Use greedy decoding (num_beams=1) and short max_length for speed in this demo
            with torch.no_grad():
                out = self.model.generate(**inputs, max_length=30, num_beams=3, do_sample=False)
            
            caption = self.processor.decode(out[0], skip_special_tokens=True)
            captions.append(caption)

        return pa.array(captions)

### Adding and Backfilling Embedding and Caption Columns

Now, let's add our new generators as virtual columns and then backfill them.

In [ ]:
for column in ["image_embedding", "caption", "caption_embedding"]:
    if (column in table.schema.names):
        table.drop_columns([column])
        
table.add_columns({
    "image_embedding": GenImageEmbeddings(),
    "caption": GenCaptions(),
})

In [ ]:
import logging
import sys
logging.basicConfig(level=logging.INFO, stream=sys.stderr, force=True)

In [ ]:
with db.local_ray_context():
    table.backfill("image_embedding", checkpoint_size=CHECKPOINT_SIZE, concurrency=CONCURRENCY)

In [ ]:
# This may take a few minutes if you're running on a CPU.
with db.local_ray_context():
    table.backfill("caption", checkpoint_size=CHECKPOINT_SIZE, concurrency=CONCURRENCY)

In [ ]:
table.search().limit(5).to_pandas()

### Features on Features

Of course, feature engineering workflows often include chains of features: features that depend on other features we've already computed! Let's make some text embeddings for those captions we just generated.

In [ ]:
@udf(version="0.1", num_gpus=1 if torch.cuda.is_available() else 0, data_type=pa.list_(pa.float32(), 512))
class GenTextEmbeddings(Callable):
    def __init__(self):
        self.is_loaded=False

    def setup(self):
        self.model, _, self.preprocess = open_clip.create_model_and_transforms("ViT-B-32", pretrained="laion2b_s34b_b79k")
        self.tokenizer = open_clip.get_tokenizer("ViT-B-32")
        self.device = "cuda" if torch.cuda.is_available() else "cpu"
        self.model = self.model.to(self.device).eval()

    def __call__(self, caption: pa.Array) -> pa.Array:
        if not self.is_loaded:
            self.setup()

        embeddings = []
        for this_caption in caption:
            # Convert PyArrow scalar to Python string
            caption_str = this_caption.as_py() if hasattr(this_caption, 'as_py') else str(this_caption)
            # Tokenizer expects a list of strings, not a single string
            tokens = self.tokenizer([caption_str])
            tokens = tokens.to(self.device)
            with torch.no_grad():
                embeddings.append(self.model.encode_text(tokens).squeeze().cpu().numpy().astype(np.float32))

        return pa.array(embeddings)


In [ ]:
if "caption_embedding" in table.schema.names:
    table.drop_columns(["caption_embedding"])
    
table.add_columns({
    "caption_embedding": GenTextEmbeddings(),
})
with db.local_ray_context():
    table.backfill("caption_embedding", checkpoint_size=CHECKPOINT_SIZE, concurrency=CONCURRENCY)

In [ ]:
table.search().limit(5).to_pandas()

## 4. Updating

Let's add some new clothes to our table and rerun the backfills to add all our derived features. This will only recompute our backfills on the new rows. This doesn't save that much time in this tutorial, but it absolutely does in production. Imagine adding new data daily; you won't want to recompute your costly features on all your data every day!

We'll add another batch of data that's half as big as the first. Then, as we do the following backfills, you will notice this as the progress bars only process the new data. For example, if our dataset was originally 100 rows, and we add 50 more, progress bars will show "100/150", reflecting that the original 100 rows have already been computed.

In [ ]:
new_rows = list(generate_rows(df, IMG_DIR, start=DATASET_SIZE, end=int(DATASET_SIZE * 1.5)))
table.add(new_rows)
with db.local_ray_context():
    table.backfill("color_tags", checkpoint_size=CHECKPOINT_SIZE, concurrency=CONCURRENCY)
    table.backfill("image_embedding", checkpoint_size=CHECKPOINT_SIZE, concurrency=CONCURRENCY)
    table.backfill("caption", checkpoint_size=CHECKPOINT_SIZE, concurrency=CONCURRENCY)
    table.backfill("caption_embedding", checkpoint_size=CHECKPOINT_SIZE, concurrency=CONCURRENCY)

In [ ]:
table.to_pandas()

## 5. Wrapping up

That's the basics of feature engineering! Next, check out our other [feature engineering tutorials](https://docs.lancedb.com/tutorials/feature-engineering), or if you're ready to start building features in production, read about [Execution Contexts](https://docs.lancedb.com/geneva/jobs/contexts).